In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow.keras.backend as K
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.metrics import f1_score
from numpy.random import seed

In [3]:
from utils import DataLoader
from encoders import KMerEncoder

In [4]:
encoder = KMerEncoder(2, 200, 'random')

In [5]:
dataloader = DataLoader(
        '../datasets/x_train.fasta', 
        '../datasets/x_val.fasta', 
        '../datasets/x_test.fasta')

In [6]:
train_seq, train_labels = dataloader.generate_training_data()
test_seq, test_labels = dataloader.generate_testing_data()
val_seq, val_labels = dataloader.generate_validation_data()

In [7]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)
num_classes = len(np.unique(train_labels))

train_labels_num = le.transform(train_labels)
val_labels_num = le.transform(val_labels)
test_labels_num = le.transform(test_labels)

train_labels_bin = keras.utils.to_categorical(train_labels_num, num_classes)
val_labels_bin = keras.utils.to_categorical(val_labels_num, num_classes)
test_labels_bin = keras.utils.to_categorical(test_labels_num, num_classes)

In [8]:
y_true = np.argmax(test_labels_bin, axis=1)
y_true = le.inverse_transform(y_true)
labels,lcounts = np.unique(y_true,return_counts=True)

In [9]:
train_seq = list(map(encoder.encode, train_seq))
test_seq = list(map(encoder.encode, test_seq))
val_seq = list(map(encoder.encode, val_seq))

train_seq = keras.utils.to_categorical(train_seq)
val_seq = keras.utils.to_categorical(val_seq)
test_seq = keras.utils.to_categorical(test_seq)

In [10]:
from models import buildCNNModel

tf.keras.backend.clear_session()

batch_size = 32
nlayers = 3
learning_rate = 0.001
epochs = 15

m = buildCNNModel(inshape=train_seq.shape[1:], num_classes=num_classes, nlayers=nlayers, cnndim=1)
                
m.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

m.fit(train_seq, 
    train_labels_bin,
    verbose = 1,
    batch_size = batch_size,
    shuffle = True,
    epochs = epochs,
    validation_data = (val_seq, val_labels_bin))

Train on 105864 samples, validate on 25342 samples
Epoch 1/15
105864/105864 [==============================] - 414s 4ms/sample - loss: 0.8357 - acc: 0.7925 - val_loss: 1.0423 - val_acc: 0.7903
Epoch 2/15
105864/105864 [==============================] - 339s 3ms/sample - loss: 0.2533 - acc: 0.9346 - val_loss: 0.8710 - val_acc: 0.8311
Epoch 3/15
 13280/105864 [==>...........................] - ETA: 4:55 - loss: 0.1826 - acc: 0.9520

KeyboardInterrupt: 

In [ ]:
pred = m.predict(test_seq, verbose=1)
predicted = np.argmax(pred, axis=1)
y_pred = le.inverse_transform(predicted)